![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Disponibilización de modelos

En este notebook aprenderá a guardar un modelo y a disponibilizarlo como una API con la librería Flask. Una API (interfaz de programación de aplicaciones) es un conjunto de definiciones y protocolos que permiten que servicios, en este caso modelos, retornen resultados y respuestas sin necesidad de saber cómo están implementados.

## Instrucciones Generales:

#### Este notebook va a se modificado para realizar los pasos necesarios para guardar un modelo para luego disponibilizarlo como una API.

Este notebook esta compuesto por dos secciones. En la primera secciónn, usted beberá entrenar y guardar (exportar) un modelo de random forest para predecir si una URL es phishing (fraudulenta) o no. En la segunda parte, usará el modelo entrenado y lo disponibilizara usando la libreria *Flask*. En el siguente paper puede conocer más detalles de la base de datos que usaremos y del problema: *A. Correa Bahnsen, E. C. Bohorquez, S. Villegas, J. Vargas, and F. A. Gonzalez, “Classifying phishing urls using recurrent neural networks,” in Electronic Crime Research (eCrime), 2017 APWG Symposium on. IEEE, 2017, pp. 1–8*. https://albahnsen.files.wordpress.com/2018/05/classifying-phishing-urls-using-recurrent-neural-networks_cameraready.pdf
  
Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook. 


## Importar base de datos y librerías

In [ ]:
%pip install -r requirements.txt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importación librerías
import pandas as pd
import numpy as np

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import joblib
import ast

import os
os.chdir('..')

## Preprocesamiento

In [ ]:
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)

In [ ]:
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
try:
    WordNetLemmatizer().lemmatize('test')
except LookupError:
    nltk.download('wordnet')
try:
    nltk.word_tokenize("example")
except LookupError:
    nltk.download('punkt')
    nltk.download('punkt_tab')

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_text_1(text):
    if isinstance(text, str): # Asegurarse de que la entrada sea una cadena
        text = text.lower()
        text = re.sub(r'[^a-z\s]', '', text) # Liempieza de caracteres diferentes a letras, sustitucion por " "
        tokens = nltk.word_tokenize(text)
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 2] # Lematización y eliminación de stopwords y palabras cortas
        return ' '.join(tokens)
    return '' # Manejar casos donde el texto no es una cadena

In [ ]:
dataTraining['plot_clean_1'] = dataTraining['plot'].apply(preprocess_text_1)

## Entrenar y guardar el modelo

In [ ]:
vect = CountVectorizer(max_features=1000)
X_dtm_1 = vect.fit_transform(dataTraining['plot_clean_1'])

In [ ]:
dataTraining['genres_list'] = dataTraining['genres'].apply(ast.literal_eval)

In [ ]:
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(dataTraining['genres_list'])

genre_labels = mlb.classes_

# Crear nuevo DataFrame con columnas binarias
genre_df = pd.DataFrame(genre_matrix, columns=genre_labels)

# Unir al dataframe original si lo deseas
dataTraining_bin = pd.concat([dataTraining.reset_index(drop=True), genre_df], axis=1)

In [ ]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))  # convertir a lista
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [ ]:
clf = OneVsRestClassifier(LogisticRegression(n_jobs=-1, max_iter=1000, random_state=42))
clf.fit(X_dtm_1, y_genres)

In [ ]:
# Exportar modelo a archivo binario .pkl
joblib.dump(clf, 'genres_movie.pkl', compress=3)

Prueba: Para probar el modelo vamos a crear una funcion dentor de un .py

In [ ]:
# Datos de ejemplo para predecir
X_test = "As a group of first responders escape the clutches of death, they begin to be killed by increasingly improbable and murderous mishaps."

In [2]:
# Definir función de predicción usando los objetos ya cargados en el notebook
import sys
sys.path.append(r'C:\Users\CYBER\Documents\Maestria MIAD\Maestria-MIAD\Machine Learning MIAD\MIAD_ML_NLP_2025-1\Entregas_ML_NLP\Semana 7\AWS')
from Proyecto2_model_deployment_P2 import predict_genre, preprocess_text_1

In [3]:
# Preprocesar y vectorizar el texto antes de predecir
predict_genre([X_test])

c:\Users\CYBER\Documents\Maestria MIAD\Maestria-MIAD\Machine Learning MIAD\MIAD_ML_NLP_2025-1\Entregas_ML_NLP\.venv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.4.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\CYBER\Documents\Maestria MIAD\Maestria-MIAD\Machine Learning MIAD\MIAD_ML_NLP_2025-1\Entregas_ML_NLP\.venv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.4.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\CYBER\Documen

ValueError: Iterable over raw text documents expected, string object received.